In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

2023-10-25 11:45:47.249257: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

In [3]:
def to_trainsets(features, labels):
    features = features.rename(columns={"date_forecast" : "time"})
    trainset = pd.merge(features, labels, how="inner", on=["time"])
    trainsetX = trainset.iloc[:,2:-1]
    trainsetY = trainset.iloc[:,-1]
    trainsetX = trainsetX.fillna(method='ffill')
    trainsetX = trainsetX.fillna(method='bfill')
    trainsetY = trainsetY.fillna(method='ffill')
    trainsetY = trainsetY.fillna(method='bfill')
    return (trainsetX, trainsetY)

In [4]:
(a_trainsetX, a_trainsetY) = to_trainsets(pd.concat([X_train_estimated_a, X_train_observed_a]), train_a)
(b_trainsetX, b_trainsetY) = to_trainsets(pd.concat([X_train_estimated_b, X_train_observed_b]), train_b)
(c_trainsetX, c_trainsetY) = to_trainsets(pd.concat([X_train_estimated_c, X_train_observed_c]), train_c)
trainsetX = pd.concat([a_trainsetX, b_trainsetX, c_trainsetX])
trainsetY = pd.concat([a_trainsetY, b_trainsetY, c_trainsetY])
print(a_trainsetY.size)
print(b_trainsetY.size)
print(c_trainsetY.size)
trainsetY


/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetX = trainsetX.fillna(method='ffill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetX = trainsetX.fillna(method='bfill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetY = trainsetY.fillna(method='ffill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() in

34061
32823
32130


/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetX = trainsetX.fillna(method='ffill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetX = trainsetX.fillna(method='bfill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetY = trainsetY.fillna(method='ffill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_25858/3278019508.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() in

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
32125    0.0
32126    0.0
32127    0.0
32128    0.0
32129    0.0
Name: pv_measurement, Length: 99014, dtype: float64

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(45, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1)
])
model.compile(
  loss='mean_squared_error'
  )

In [6]:
history = model.fit(trainsetX, trainsetY, batch_size=1000, epochs=1000)

Epoch 1/1000
100/100 [==============================] - 2s 6ms/step - loss: 230450352.0000
Epoch 2/1000
100/100 [==============================] - 1s 5ms/step - loss: 51470192.0000
Epoch 3/1000
100/100 [==============================] - 1s 6ms/step - loss: 17268808.0000
Epoch 4/1000
100/100 [==============================] - 0s 5ms/step - loss: 4393860.5000
Epoch 5/1000
100/100 [==============================] - 0s 5ms/step - loss: 1220652.1250
Epoch 6/1000
100/100 [==============================] - 0s 5ms/step - loss: 547770.3750
Epoch 7/1000
100/100 [==============================] - 0s 5ms/step - loss: 428357.0312
Epoch 8/1000
100/100 [==============================] - 0s 5ms/step - loss: 389825.8125
Epoch 9/1000
100/100 [==============================] - 0s 4ms/step - loss: 390599.2188
Epoch 10/1000
100/100 [==============================] - 0s 5ms/step - loss: 383561.9062
Epoch 11/1000
100/100 [==============================] - 0s 5ms/step - loss: 377140.5625
Epoch 12/1000
100/100

KeyboardInterrupt: 